In [90]:
import requests as req , re , pandas as pd
from pyvis.network import Network

In [91]:
def connect(source):
    url = 'https://t.me/s/' + source
    request = req.get(url)
    html = request.text
    print('connected to' , source)
    return html

In [92]:
def targets(html , source):
    regex = re.compile(r'href=.https://t.me/\w+')
    data = []
    for target in regex.findall(html):
        target = target.rsplit('/' , 1)[1]
        if target.lower() != source.lower():
            data.append([source.lower(), target.lower()])
    edge_df = pd.DataFrame(data , columns=['source', 'target'])
    print('got targets from' , source)
    return edge_df


In [93]:
def subs (html , source):
    regex = re.compile(r'class=.counter_value.>[^<]+')
    try:
        string = regex.findall(html)[0]
        number = convert_str_to_number(string.rsplit('>' , 1)[1])
        print('got ' , number ,  'subs from ' , source)
        return number
    except:
        print('cant get subs for' , source)
    return

note: connect , targets and subs functions can be edited per social network

In [94]:
def convert_str_to_number(x):
    total_stars = 0
    num_map = {'K':1000, 'M':1000000, 'B':1000000000}
    if x.isdigit():
        total_stars = int(x)
    else:
        if len(x) > 1:
            total_stars = float(x[:-1]) * num_map.get(x[-1].upper(), 1)
    return int(total_stars)

In [95]:
def finals(source):
    html = connect(source)
    edge_df = targets(html , source)
    if source[-3:].lower() != 'bot':
        size = subs(html , source)
        edge_df['source_node_size'] = size
    edge_df['edge_size'] = edge_df.groupby(['target'])['source'].transform('count')
    edge_df = edge_df.drop_duplicates(subset=['target'])
    print('passed finals for' , source)
    return edge_df

In [96]:
def loop(df , i , start_channel_name, iter_number):
    iter = iter_number
    if i < iter:
      targets = df['target']
      for target in targets:
          edge_df = finals(target)
          df = df.append(edge_df)
          print('current length = ' , len(df))
      if i+1==iter:
        df.to_csv('{} iteration for {}.csv'.format(str(i+1),start_channel_name))
      i += 1
      print('iteration' , i)
      loop(df , i , start_channel_name, iter)
      return
    else: return "finished"

In [97]:
channel_name = input("Enter the channel name: ")
iterations_number = int(input("Enter the number of iterations: "))
loop(finals(channel_name) , 0 , channel_name, iterations_number)

connected to crypto
got targets from crypto
got  56000 subs from  crypto
passed finals for crypto
connected to listmebot
got targets from listmebot
passed finals for listmebot
current length =  18
connected to cryptofaqs
got targets from cryptofaqs
got  703 subs from  cryptofaqs
passed finals for cryptofaqs
current length =  26
connected to airdrops
got targets from airdrops
got  9040 subs from  airdrops
passed finals for airdrops
current length =  51
connected to cryptoalerts
got targets from cryptoalerts
got  19800 subs from  cryptoalerts
passed finals for cryptoalerts
current length =  54
connected to cryptobots
got targets from cryptobots
got  2510 subs from  cryptobots
passed finals for cryptobots
current length =  89
connected to cryptochannels
got targets from cryptochannels
got  4930 subs from  cryptochannels
passed finals for cryptochannels
current length =  151
connected to cryptocommunities
got targets from cryptocommunities
got  5110 subs from  cryptocommunities
passed fina

In [98]:
got_net = Network(height='750px', width='100%', bgcolor='#222222', font_color='whitw')

In [99]:
edge_df = pd.read_csv('{} iteration for {}.csv'.format(str(iterations_number), channel_name))

edge_data = zip(edge_df['source'], edge_df['target'], edge_df['edge_size'])

FileNotFoundError: [Errno 2] No such file or directory: '0 iteration for crypto.csv'

In [ ]:
for row in edge_data:
    src = row[0]
    trgt = row[1]
    w = row[2]
    node_link = "<a href=\ 'http://t.me/s/" + src + "'>" + src + "</a>"
    got_net.add_node(src, src, title=node_link)
    got_net.add_node(trgt, trgt, title=trgt)
    got_net.add_edge(src, trgt, value=w)

neighbor_map = got_net.get_adj_list()
# add neighbor data to node hover data
for node in got_net.nodes:
    node['title'] += ' Neighbors:<br>' + '<br>'.join(neighbor_map[node['id']])
    node['value'] = len(neighbor_map[node['id']])

In [ ]:
got_net.show_buttons()
got_net.show('{} iteration for {}.html'.format(str(iterations_number-1),channel_name))